In [1]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import time
import csv

In [2]:
rootURL = 'https://www.mycareersfuture.sg'
#mainURL = 'https://www.mycareersfuture.sg/search?sortBy=new_posting_date&page=0'

In [7]:
# Go the specific page and retrieve the number of jobs
initialURL = 'https://www.mycareersfuture.sg/search?sortBy=new_posting_date&page=0'

# call up the Chrome webdriver
driver = webdriver.Chrome(executable_path = 'C:/Users/nchandra.CATLIN/chromedriver')
driver.get(initialURL)

time.sleep(5)

html = driver.page_source
firstsoup = bs(html, 'lxml')
    
time.sleep(1)
    
driver.close()


In [9]:
jobcount = firstsoup.find('span', {'id':'search-results-message'}).find('div').find(text = True, recursive = False)
jobcount

'20,260 jobs found'

In [10]:
pagecount = round(int(jobcount.split()[0].replace(',',''))/20)
pagecount

1013

In [15]:
# loop through all the pages and return the fields specified
records = []
for pcount in range(pagecount):
#for pcount in range(1,10):
    
    # Go the specific page
    searchURL = 'https://www.mycareersfuture.sg/search?sortBy=new_posting_date&page='+str(pcount)
    
    driver = webdriver.Chrome(executable_path = 'C:/Users/nchandra.CATLIN/chromedriver')
    driver.get(searchURL)

    time.sleep(5)

    html = driver.page_source
    soup = bs(html, 'lxml')
    
    time.sleep(1)
    
    driver.close()
    
    for x in range(20):
        company = ''
        jobtitle = ''
        location = ''
        jobtype = ''
        seniority = ''
        industry = ''
        salary = ''
        govtscheme = ''
        skill_list = ''
        jd_list = ''
        jr_list = ''
        
        cardno = 'job-card-' + str(x)
        
        #link.append(soup.find('div', {'id':cardno}).find('div', {'class':'card relative'}).find('a')['href'])
        
        # Create job detail link
        try:
            sublink = soup.find('div', {'id':cardno}).find('div', {'class':'card relative'}).find('a')['href']
        except AttributeError:
            print('cannot find next link')
            continue
        else:
            jobdetaillink = rootURL + sublink
            
        ## Open job detail page
        driver = webdriver.Chrome(executable_path = 'C:/Users/nchandra.CATLIN/chromedriver')
        driver.get(jobdetaillink)

        time.sleep(5)

        jobdetailpage_html = driver.page_source
        jobdetailsoup = bs(jobdetailpage_html, 'lxml')
    
        time.sleep (5)
        
        driver.close()
    
        # Company name
        co = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('p', {'name':'company'} )
        
        if co is not None:
            company= co.getText().strip()
            
        else:
            company = ''
        
        #Job Title
        jobt = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('h1', {'name':'job_title'})
        
        if jobt is not None:
            jobtitle = jobt.getText().strip()
        else:
            jobtitle = ''
    
    
        # Location
        loc = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('section', {'name' : 'job_info'}).find('p', {'name':'location'})
    
        if loc is not None:
            location = loc.text.strip()
        else:
            location = ''
      
    
    # Job Type
        job = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('p', {'name':'employment_type'})
    
        if job is not None:
            jobtype = job.getText().strip()

        else:
            jobtype = ''
    
    # Seniority
        snr = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('p', {'name':'seniority'})
    
        if snr is not None:
            seniority = snr.getText().strip()
        else:
            seniority = ''
        
    # Industry
        ind = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('p', {'name':'category'})
        
        if ind is not None:
            industry = ind.getText().strip()
        else:
            industry = ''
       
    # Salary
        sal = soup.find('div', {'id': cardno}).find('div', {'class':'card relative'}).find('a').find('span', {'class':'salary_range'})
        
        if sal is not None:
            salary = sal.text.strip()
        else:
            salary = ''

    
    # Government Support
        gscheme = soup.find('div', {'id':cardno}).find('div', {'class':'card relative'}).find('a').find('p', {'name':'schemes_available'})
        
        if gscheme is not None:
            govtscheme = gscheme.text.strip()
            
        else:
            govtscheme = ''
    
        
    ## Get skill
        skill_cont = []
        
        
        skill_finder = jobdetailsoup.find('div',{'class': 'multi-pill-button'})
        
        if skill_finder is not None:
             for skill in skill_finder.find_all('label'):
                skill_cont.append(skill.text.strip())           

        else:
            skill_cont.append('')

        skill_list = skill_cont
        
    ## Get job description
        jd_cont = []
        
        jd_finder = jobdetailsoup.find('div', {'id':'job_description'})
        
        if jd_finder is not None:
            for jd in jd_finder.find_all(['li', 'p']):
                jd_cont.append(jd.text.strip())             
            
        else:
            jd_cont.append('')
        
        jd_list = jd_cont
        
    ## Get job requirement
        jr_cont = []
        
        jr_finder = jobdetailsoup.find('div', {'id':'requirements'})
        
        try:
            for jr in jr_finder.find_all(['li', 'p']):
                jr_cont.append(jr.text.strip())
        except:
            jr_cont.append('')
        jr_list = jr_cont
        
        records.append([company, jobtitle, location, jobtype, seniority, industry, salary, govtscheme, skill_list, jd_list, jr_list])



In [16]:
len(records)

180

In [19]:
# write to CSV
with open("C:/Users/nchandra.CATLIN/Desktop/output.csv", "a", encoding='utf-8-sig') as f:
    writer = csv.writer(f, lineterminator = '\n')

    for item in records:
        writer.writerow(item)

In [ ]:
skill_list

In [ ]:
len(jr_list)

In [ ]:
len(company)

In [ ]:
# to create a dataframe out of the scraped data

#df = pd.DataFrame(
    #{'Company': company,
    #'Job_Title':jobtitle,
     #'Location': location,
     #'Seniority': seniority,
     #'Industry': industry,
     #'Salary': salary,
     #'Govt_Scheme': govtscheme,
     #'Skills':skill_list,
     #'Job_Desc': jd_list,
     #'Job_Req':jr_list
    #}
#)

In [ ]:
#df.iloc[13:,:]

In [ ]:
len(df)